## Tensor的创建


In [3]:
# torch.tensor()函数的使用
import torch

a = torch.tensor([[0.11111, 0.222222, 0.3333333]],
        dtype=torch.float64,
        device=torch.device('cuda:0'),
        requires_grad=True)
a


tensor([[0.1111, 0.2222, 0.3333]], device='cuda:0', dtype=torch.float64,
       requires_grad=True)

In [4]:
# torch.from_numpy()函数的使用.tensor与numpy数组共享内存
import numpy as np
a = np.array([1, 2, 3])
t = torch.from_numpy(a)
print(t)
t[0] = -1
a

tensor([1, 2, 3], dtype=torch.int32)


array([-1,  2,  3])

In [7]:
# zeros, ones。t和out_t共享内存
out_t = torch.zeros(2, 3)
t = torch.zeros((3, 3), out=out_t)
print(t, '\n', out_t)
print(id(t), id(out_t))

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]) 
 tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
1788577350864 1788577350864


In [8]:
# zeros_like ones_like生成与输入相同形状的tensor
input = torch.empty(2, 3)
x = torch.ones_like(input)
x

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [10]:
# full：生成指定值的tensor；full_like：生成与输入相同形状的tensor
# empty：生成未初始化的tensor；empty_like：生成与输入相同形状的未初始化tensor
y = torch.full((2, 3), 3.141592)
z = torch.full_like(input, 1234)
print(y, '\n', z)
m = torch.empty((2,3), dtype=torch.int64)
n = torch.empty_like(m, dtype=torch.float32)
print(m, '\n', n)

tensor([[3.1416, 3.1416, 3.1416],
        [3.1416, 3.1416, 3.1416]]) 
 tensor([[1234., 1234., 1234.],
        [1234., 1234., 1234.]])
tensor([[0, 0, 0],
        [0, 0, 0]]) 
 tensor([[3.2338e-27, 1.8553e-42, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00]])


In [17]:
# arange：生成等差数列；linspace：生成等分数列
# linspace: 生成等分数列
# logspace: 生成等比数列
x = torch.arange(1, 4, 0.5)
y = torch.linspace(1, 4, 2)
z = torch.logspace(0, 1, 2)
x, y, z


(tensor([1.0000, 1.5000, 2.0000, 2.5000, 3.0000, 3.5000]),
 tensor([1., 4.]),
 tensor([ 1., 10.]))

In [18]:
# eye:生成单位矩阵；diag:生成对角矩阵
x = torch.eye(3)
y = torch.diag(torch.tensor([1, 2, 3]))
x, y

(tensor([[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]),
 tensor([[1, 0, 0],
         [0, 2, 0],
         [0, 0, 3]]))

In [25]:
# 根据概率分布创建张量
# normal：正态分布；poisson：泊松分布；exponential：指数分布；bernoulli：伯努利分布
# geometric：几何分布；log_normal：对数正态分布；cauchy：柯西分布
x_1 = torch.normal(mean=torch.arange(1, 5).float(), std=torch.arange(1, 5).float())
x_2 = torch.normal(mean=0.5, std=1.0, size=(10,))
# 均匀分布，[0, 1)
y_1 = torch.rand(2,3)
y_2 = torch.rand_like(y_1)
y_3 = torch.randint(0, 10, (2, 3))
x_1, x_2, y_1, y_2, y_3

(tensor([-0.5442,  4.8789, -2.0258,  1.9685]),
 tensor([-1.4930, -0.0703, -1.2334,  2.1170,  1.5179, -0.5595, -1.1406,  1.6284,
         -0.2623, -0.2705]),
 tensor([[0.2928, 0.4948, 0.4199],
         [0.8191, 0.9728, 0.2496]]),
 tensor([[0.9647, 0.4886, 0.2223],
         [0.7573, 0.6296, 0.2102]]),
 tensor([[7, 2, 7],
         [8, 7, 1]]))

## 张量操作与线性回归


# 张量元数据：大小(size)、偏移(offset)和步长(stride)

In [26]:
points = torch.zeros(3, 2)
points.shape

torch.Size([3, 2])

In [27]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [28]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [29]:
points[0, 1]

tensor(1.)

In [30]:
points[0]

tensor([4., 1.])

In [31]:
# 张量的存储是由一个存储器（storage）对象来管理的。存储器对象实际上是一块连续的内存区域，包含了张量中的所有元素。
# 对于给定的张量，可以通过调用其 .storage() 方法来访问其存储器对象。
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

C:\Users\xiangzy\AppData\Local\Temp\ipykernel_9412\1675611638.py:4: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [32]:
points_storage = points.storage()
points_storage[0]

4.0

In [33]:
points.storage()[1]

1.0

In [34]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

In [35]:
# storage_offset() 方法用于返回张量在其存储器对象中的偏移量。
# oints 张量的存储器对象中的元素按行优先顺序排列，而第二个元素（索引为 1）位于存储器对象中的第 2 个位置。
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [36]:
second_point.size() # 张量的 size() 方法用于返回张量的大小（即各个维度的长度）。

torch.Size([2])

In [37]:
second_point.shape  #张量的 shape 属性用于返回张量的形状（即各个维度的长度）

torch.Size([2])

In [38]:
second_point = points[1]
second_point.size()

torch.Size([2])

In [39]:
second_point.storage_offset()

2

In [24]:
second_point.stride()   #张量的 stride() 方法用于返回张量在存储器对象中沿着每个维度的跨度

(1,)

In [40]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [41]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()    #赋值，不使用同一片内存，否则一个修改另一个也修改
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

: 

In [27]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [28]:
points_t = points.t()   # 矩阵转置
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [29]:
id(points.storage()) == id(points_t.storage())

True

In [30]:
points.stride()

(2, 1)

In [31]:
points_t.stride()

(1, 2)

In [32]:
some_t = torch.ones(3, 4, 5)    #4X5, 4, 1
some_t.stride()

(20, 5, 1)

In [33]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)    #0与2换
some_t.shape

torch.Size([3, 4, 5])

In [34]:
transpose_t.shape

torch.Size([5, 4, 3])

In [35]:
some_t.stride()

(20, 5, 1)

In [36]:
transpose_t.stride()    #transpose后stride变化

(1, 5, 20)

In [37]:
points.is_contiguous()  #内存是否连续

True

In [38]:
points_t.is_contiguous()

False

In [39]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [40]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [41]:
points_t.stride()

(1, 2)

In [42]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [43]:
points_t_cont.stride()

(3, 1)

In [44]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

# 张量元素类型

In [45]:
#  使用dtype指定数值类型
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [46]:
short_points.dtype

torch.int16

In [47]:
# 相应的强制转换方法将张量构造函数的输出强制转换为正确的类型
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [48]:
# 方法to
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

In [62]:
# 向下取整
a = torch.tensor(12.99)
a.to(torch.short)

tensor(12, dtype=torch.int16)

In [60]:
points_64 = torch.rand(5, dtype=torch.double)  # 长度为5的随机张量（0-1）
print(points_64)
points_short = points_64.to(torch.short)
print(points_short)
points_64 * points_short  # works from PyTorch 1.3 onwards

tensor([0.0508, 0.7023, 0.7161, 0.0543, 0.6920], dtype=torch.float64)
tensor([0, 0, 0, 0, 0], dtype=torch.int16)


tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

# 索引张量

In [63]:
some_list = list(range(6)) 
some_list[:]               #<-----所有元素都在列表中
some_list[1:4]             #<-----从元素1(含)到元素4(不含)
some_list[1:]              #<-----从元素1(含)到列表的末尾
some_list[:4]              #<-----从列表的开始到元素4(不含)
print(some_list[:-1])             #<-----从列表的开始到最后一个元素的前一个
some_list[1:4:2]           #<-----从元素1(含)到元素4(不含)，以2为步长

[0, 1, 2, 3, 4]


[1, 3]

In [66]:
# reset points back to original value 将点重置回原始值
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [69]:
print(points[1:])          #<-------第一行之后的所有行；隐式选择所有列
print(points[1:, :])       #<-------第一行之后的所有行；所有列
print(points[1:, 0])       #<-------第一行之后的所有行；第一列
print(points[None])        #<-------增加一个尺寸为1的维度，就像unsqueeze一样
print(points[None].shape)

tensor([[5., 3.],
        [2., 1.]])
tensor([[5., 3.],
        [2., 1.]])
tensor([5., 2.])
tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])
torch.Size([1, 3, 2])


In [53]:
# tensor转为numpy
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [54]:
# # numpy转为tensor
points = torch.from_numpy(points_np)

# 读写文件

In [70]:
# 存储文件
torch.save(points, '../data/p1ch3/ourpoints.t')

In [56]:
# 打开文件进行读写
with open('../data/p1ch3/ourpoints.t','wb') as f:
   torch.save(points, f)

In [57]:
# 加载文件数据
points = torch.load('../data/p1ch3/ourpoints.t')

In [58]:
with open('../data/p1ch3/ourpoints.t','rb') as f:
   points = torch.load(f)

In [71]:
import h5py #读写大号网络/模型
# 创建一个名为'ourpoints.hdf5'的HDF5文件，并在其中创建了一个名为'coords'的数据集，该数据集保存了一个Numpy数组points的数据。
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

In [76]:
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'r')
dset = f['coords']
print(dset)
last_points = dset[:]
last_points

<HDF5 dataset "coords": shape (3, 2), type "<f4">


array([[4., 1.],
       [5., 3.],
       [2., 1.]], dtype=float32)

In [77]:
last_points = torch.from_numpy(dset[:])
print(last_points)
f.close()

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])


In [78]:
# 在GPU里定义
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')

In [79]:
# GPU + CPU不行
# 预期所有张量都在同一设备上，但至少找到了两个设备，cuda:0和cpu！
points_gpu + points

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [80]:
# 转为GPU
points = points.to(device='cuda')
points_gpu + points

tensor([[ 8.,  2.],
        [10.,  6.],
        [ 4.,  2.]], device='cuda:0')

In [ ]:
# 同理
points_gpu = points.to(device='cuda:0')

In [82]:
points_gpu = points_gpu + 4
points_gpu

tensor([[12.,  9.],
        [13., 11.],
        [10.,  9.]], device='cuda:0')

In [83]:
# 转CPU
points_cpu = points_gpu.to(device='cpu')

In [84]:
# points.cuda(0) 将张量 points 移动到指定编号为 0 的GPU设备上。如果你有多个GPU设备，并且想要使用特定的设备进行计算，可以通过设置相应的编号来指定。
# points_gpu.cpu() 将之前移动到GPU上的张量 points_gpu 移回到CPU上。这通常是在需要对数据进行后续处理或输出时使用。
points_gpu = points.cuda(0)
points_cpu = points_gpu.cpu()

In [ ]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [ ]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [ ]:
a = torch.ones(3, 2)

In [ ]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])